In [ ]:
from pyspark import SparkContext

In [ ]:
import os

In [ ]:
os.environ["PYTHONHASHSEED"] = "5757"

## Данные в Spark разбиваются на партиции. Партиция - это атомарная единица параллелизма. Но как именно происходит присваивание записи в RDD партиции на экзекьюторе?

![](pics/lesson01_02_shuffle_partitioning.png)

## Разбиение происходит с помощью объекта `partitioner`, который имеет следующий интерфейс:
* numPartitions - число партиций в RDD
* getPartition - возвращает соответствие ключа индексу партиции

In [ ]:
nums = list(range(10))
nums

In [ ]:
with SparkContext(master="local[1]") as sc:
    rdd = sc.parallelize(nums)
    
    print("Number of partitions: {}".format(rdd.getNumPartitions()))
    print("Partitioner: {}".format(rdd.partitioner))
    print("Partitions structure: {}".format(rdd.glom().collect()))

## В случае отстутствия партишенера, разбиение данных не зависит от самих данных. Данные разбиваются равномерно, исходя из размера 

In [ ]:
with SparkContext("local[2]") as sc:
    rdd = sc.parallelize(nums)
    
    print("Default parallelism: {}".format(sc.defaultParallelism))
    print("Number of partitions: {}".format(rdd.getNumPartitions()))
    print("Partitioner: {}".format(rdd.partitioner))
    print("Partitions structure: {}".format(rdd.glom().collect()))

In [ ]:
nums = [4, 3, 2, 1, 0, 9, 8, 7, 6, 5]

In [ ]:
with SparkContext("local[2]") as sc:
    rdd = sc.parallelize(nums)
    
    print("Default parallelism: {}".format(sc.defaultParallelism))
    print("Number of partitions: {}".format(rdd.getNumPartitions()))
    print("Partitioner: {}".format(rdd.partitioner))
    print("Partitions structure: {}".format(rdd.glom().collect()))

## Что произойдет, если партиций больше, чем данных?

In [ ]:
with SparkContext("local[1]") as sc:
    rdd = sc.parallelize(nums, 15)
    
    print("Number of partitions: {}".format(rdd.getNumPartitions()))
    print("Partitioner: {}".format(rdd.partitioner))
    print("Partitions structure: {}".format(rdd.glom().collect()))

In [ ]:
sc = SparkContext("local[1]")
rdd = sc.parallelize(nums, 15)

print("Number of partitions: {}".format(rdd.getNumPartitions()))
print("Partitioner: {}".format(rdd.partitioner))
print("Partitions structure: {}".format(rdd.glom().collect()))

In [ ]:
sc.stop()

## Партишенеры работают с PairRDD

In [ ]:
sc = SparkContext("local[2]")

![](pics/lesson01_02_shuffle_partitioning_02.png)

In [ ]:
rdd = sc.parallelize(nums) \
        .map(lambda el: (el, el)) \
        .partitionBy(2) \
        .persist()
    
print("Number of partitions: {}".format(rdd.getNumPartitions()))
print("Partitioner: {}".format(rdd.partitioner))
print("Partitions structure: {}".format(rdd.glom().collect()))

In [ ]:
rdd.partitioner.partitionFunc(8) % 2

## По-умолчанию, Spark использует HashPartitioner

In [ ]:
from pyspark.rdd import portable_hash

num_partitions = 2
for el in nums:
    print("Element: [{}]: {} % {} = partition {}".format(
        el, portable_hash(el), num_partitions, portable_hash(el) % num_partitions))

In [ ]:
portable_hash("Hello")

In [ ]:
sc.stop()

## Более реалистичный пример

In [ ]:
transactions = [
    {'name': 'Bob', 'amount': 100, 'country': 'United Kingdom'},
    {'name': 'James', 'amount': 15, 'country': 'United Kingdom'},
    {'name': 'Marek', 'amount': 51, 'country': 'Poland'},
    {'name': 'Johannes', 'amount': 200, 'country': 'Germany'},
    {'name': 'Paul', 'amount': 75, 'country': 'Poland'},
]

In [ ]:
def country_partitioner(country):
    mapping = {
        "United Kingdom": 0,
        "Poland": 1,
        "Germany": 2
    }
    return mapping.get(country, 99999)

In [ ]:
with SparkContext("local[2]") as sc:
    rdd = sc.parallelize(transactions) \
            .map(lambda el: (el['country'], el)) \
            .partitionBy(3, country_partitioner)
    
    print("Number of partitions: {}".format(rdd.getNumPartitions()))
    print("Partitioner: {}".format(rdd.partitioner))
    print("Partitions structure: {}".format(rdd.glom().collect()))

In [ ]:
with SparkContext("local[2]") as sc:
    rdd = sc.parallelize(transactions) \
            .map(lambda el: (el['country'], el)) \
            .partitionBy(3)
    
    print("Number of partitions: {}".format(rdd.getNumPartitions()))
    print("Partitioner: {}".format(rdd.partitioner))
    print("Partitions structure: {}".format(rdd.glom().collect()))

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
with SparkSession.builder.master("local[2]").getOrCreate() as spark:
    df = spark.createDataFrame(transactions).repartition(3, "country")
    print("Number of partitions: {}".format(df.rdd.getNumPartitions()))
    print("Partitioner: {}".format(df.rdd.partitioner))
    print("Partitions structure: {}".format(df.rdd.glom().collect()))

## Правильное использование партишенеров может сильно ускорить выполнение ваших программ!

## Важным понятием является "known partitioner", т.е. явно заданный партишенер. По-умолчанию, партишенер не задается

In [ ]:
keys = ['a'] * 1000 + ['b'] * 1000 + ['c'] * 1000 + ['d'] * 1000
values_left = [1] * 1000 + [2] * 1000 + [3] * 1000 + [4] * 1000
values_right = [5] * 1000 + [6] * 1000 + [7] * 1000 + [8] * 1000

In [ ]:
from pyspark import SparkConf

In [ ]:
conf = SparkConf().set("spark.default.parallelism", "10")

In [ ]:
sc = SparkContext(appName="Optimization", conf=conf)

In [ ]:
left_rdd = sc.parallelize(zip(keys, values_left))

In [ ]:
left_rdd.getNumPartitions()

In [ ]:
left_rdd.partitioner

In [ ]:
left_rdd.take(5)

In [ ]:
right_rdd = sc.parallelize(zip(keys, values_right))

In [ ]:
right_rdd.getNumPartitions()

In [ ]:
right_rdd.partitioner

In [ ]:
right_rdd.take(5)

In [ ]:
left_rdd.join(right_rdd).count()

In [ ]:
left_rdd.join(right_rdd).getNumPartitions()

In [ ]:
sc.stop()

## Если у двух RDD партишенеры явно заданы и они равны, то такие RDD называются ко-партиционированными

![](pics/lesson01_02_shuffle_partitioning_03.png)

In [ ]:
sc = SparkContext(appName="Optimization")

In [ ]:
left_rdd = sc.parallelize(zip(keys, values_left))

In [ ]:
left_rdd = left_rdd.partitionBy(4)

In [ ]:
left_rdd.partitioner

In [ ]:
left_rdd.take(5)

In [ ]:
right_rdd = sc.parallelize(zip(keys, values_right))

In [ ]:
right_rdd = right_rdd.partitionBy(4)

In [ ]:
right_rdd.partitioner

In [ ]:
right_rdd.take(5)

In [ ]:
left_rdd.partitioner == right_rdd.partitioner

In [ ]:
left_rdd.join(right_rdd).count()

In [ ]:
left_rdd.join(right_rdd).getNumPartitions()

## Сохранение партишенеров между трансформациями особенно полезно для широких трансформаций, потому что помогают снизить объем шаффла или вовсе его исключить

## Особенность, правда, заключается еще и в том, что узкие трансформации способны подгадить!

In [ ]:
left_rdd.partitioner

In [ ]:
left_rdd_identity = left_rdd.map(lambda x: x)

In [ ]:
left_rdd_identity.partitioner

In [ ]:
left_rdd_identity.partitioner == right_rdd.partitioner

In [ ]:
left_rdd_identity.join(right_rdd).count()

In [ ]:
left_rdd_identity.join(right_rdd).partitioner == right_rdd.partitioner

## Это происходить из-за того, что трансформации типа `map()` могут изменять не только значения PairRDD, но и ключи! Выходом из ситуации может быть либо использования трансформаций над значениями, например `mapValues()`, либо передача параметра `preservesPartitioning=True`

In [ ]:
left_rdd_identity = left_rdd.map(lambda x: (x[0], x[1]), preservesPartitioning=True)

In [ ]:
left_rdd_identity.partitioner

In [ ]:
left_rdd_identity.partitioner == right_rdd.partitioner

In [ ]:
left_rdd_identity.join(right_rdd).count()

In [ ]:
left_rdd_identity.join(right_rdd).partitioner == right_rdd.partitioner

In [ ]:
sc.stop()

## Arrow

![](https://arrow.apache.org/img/copy.png)
![](https://arrow.apache.org/img/shared.png)

A critical component of Apache Arrow is its in-memory columnar format, a standardized, language-agnostic specification for representing structured, table-like datasets in-memory. 
![](https://arrow.apache.org/img/simd.png)

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

In [ ]:
spark = SparkSession.builder.master("local[2]").getOrCreate()

In [ ]:
spark

In [ ]:
df = spark.range(int(1e6)).withColumn("value", f.lit("Just a random string passing by")).cache()

In [ ]:
df.count()

In [ ]:
spark.conf.set("spark.sql.execution.arrow.enabled", "false")

In [ ]:
%%time
pdf = df.toPandas()

In [ ]:
pdf

In [ ]:
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [ ]:
%%time
pdf = df.toPandas()

In [ ]:
pdf

In [ ]:
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", 10)

In [ ]:
%%time
pdf = df.toPandas()

In [ ]:
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", 100000)

In [ ]:
%%time
pdf = df.toPandas()